#. 🏦 Banking Service Agent 💵

**What's about:** Banking Customer Service Agent with RAG and Sqlite toolsfor products and policy related questions.

Dev by Jeison Robles.

In [1]:
%pip install -q openai qdrant-client docling fastembed jupyter-chat-widget

In [2]:
from jupyter_chat_widget import ChatUI
from time import sleep

chat = ChatUI()

def answers(text):
  chat.rewrite("Thinking...")
  sleep(3)
  chat.rewrite("")
  chat.append("Hello "+ text)

chat.connect(answers)


Output()

Output()

Text(value='', description='user: ')

# Prep Functions

In [3]:
import openai
import os
from google.colab import userdata


def get_oai_client():
    """
        Creates the OpenAI client.
    """
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    client = openai.OpenAI()
    messages = [{"role": "system",
                 "content": """You are a helpful banking assistant,
                  you must paraphrase the policy undertand customer questions
                   and provide easy to understand, Never can provide the explicit policy. Output in HTML."""}]
    return client, messages


def query_llm(ui, client, messages):
    """
        Query the LLM and returns a stream handle to the response.
    """
    ui.rewrite("[Generating]")
    stream = client.chat.completions.create(
        model="gpt-5.2",  # Or another suitable model
        messages=messages, # Use the message history
        stream=True, # Enable streaming
    )
    return stream


def display_response(ui, stream):
    """
        Display the LLM's response in the UI one token at a time.
    """
    ui.rewrite("")
    complete_response = ""
    for chunk in stream:
        if (content := chunk.choices[0].delta.content) and content.strip():
            complete_response += content
            ui.append(content)
    return complete_response


# Import Policy

In [4]:
import os
import requests

urls_and_filenames = [
    (
        "https://raw.githubusercontent.com/JeisonRobles/Banking-Service-Agent/main/Bank_Policy_1",
        "Bank_Policy_1.md"
    ),
]

os.makedirs("documents", exist_ok=True)

for url, filename in urls_and_filenames:
    response = requests.get(url)
    response.raise_for_status()
    with open(os.path.join("documents", filename), "wb") as f:
        f.write(response.content)

# Documents processing

In [5]:
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter
from uuid import uuid4
from qdrant_client import QdrantClient, models


def ingest_documents(ui, paths):
    print("Ingesting documents...")
    print(paths)
    # Prepare the vectordb and embedder
    vdb = QdrantClient(location=":memory:")
    dense_model = "sentence-transformers/all-MiniLM-L6-v2"
    vdb.set_model(dense_model)
    collection_name = "documents"
    vdb.create_collection(
        collection_name=collection_name,
        vectors_config=vdb.get_fastembed_vector_params(),
    )
    points = []
    for path in paths:
        print(path)

    print(f"processing path: {paths}")

    ui.rewrite(f"[Parsing {paths}...]")


    for path in paths:

        print(f"processing path: {path}")

        ui.rewrite(f"[Parsing {path}...]")

        # Parse the document with docling
        doc = DocumentConverter().convert(source=path).document

        # Chunk the document
        chunker = HybridChunker()
        chunk_iter = chunker.chunk(dl_doc=doc)

        # Enrich the chunks and build Qdrant points
        for chunk in chunk_iter:
            enriched_text = chunker.contextualize(chunk=chunk)
            meta = chunk.meta.export_json_dict()
            points.append(
                models.PointStruct(
                    id=uuid4().hex,
                    payload=meta | {"document": enriched_text},
                    vector={
                        # FastEmbed uses named vector fields derived from the model names
                        vdb.get_vector_field_name(): models.Document(text=enriched_text, model=dense_model),
                    },
                )
            )
    # Upload (embeddings happen internally because we used models.Document)
    vdb.upload_points(collection_name=collection_name, points=points, batch_size=64, wait=True)
    ui.rewrite(f"All documents were processed. I'm ready!")
    return vdb

def add_context(ui, vdb, query):
    """
        Queries the vector database for relevant context snippets
        and adds them to the LLM's context.
    """

    print("Adding context...")
    ui.rewrite("[Searching]")
    samples = vdb.query(
        collection_name="documents",
        query_text=query,
        limit=10,
    )
    ui.rewrite(f"[Found {len(samples)} relevant snippets]")
    sleep(1)
    return {
        "role": "user",
        "content": f"Relevant snippets from the document: {'\n\n'.join(s.document for s in samples)}"
    }

## 💾 SQL Local DB

Lets implement an small SQLite db for customers patterns


In [6]:
import sqlite3
from pathlib import Path

DB_PATH = Path("bank_demo.sqlite")

def init_customer_db(db_path: Path= DB_PATH):
  """Creates a tiny local SQLite DB with customers."""
  conn = sqlite3.connect(str(db_path))
  cur = conn.cursor()
  cur.execute("""
  CREATE TABLE IF NOT EXISTS customers(
    customer_id TEXT PRIMARY KEY,
    full_name TEXT NOT NULL
  )
  """)

  #Seed demo rows (idemponent)
  cur.executemany(
      "INSERT OR IGNORE INTO customers (customer_id, full_name) VALUES (?, ?)",
      [
          ("CUST-1001", "Alicia Mora"),
          ("CUST-2002", "Diego Hernandez"),
          ("CUST-3OO3", "Maria Jose Vargas"),
      ]
  )
  conn.commit()
  conn.close()
  return str(db_path.resolve())

DB_RESOLVED = init_customer_db()
print(f"SQLite demo DB ready at: {DB_RESOLVED}")

SQLite demo DB ready at: /content/bank_demo.sqlite


In [7]:

# --- Tool: lookup customer name (used only for "my products"/personal info requests) ---
import sqlite3
from typing import Dict, Any
import json

def get_customer_name(customer_id: str) -> Dict[str, Any]:
    """Return customer name for greeting; no products yet."""
    conn = sqlite3.connect(str(DB_PATH))
    cur = conn.cursor()
    cur.execute("SELECT full_name FROM customers WHERE customer_id = ?", (customer_id,))
    row = cur.fetchone()
    conn.close()
    if not row:
        return {"found": False, "customer_id": customer_id, "full_name": None}
    return {"found": True, "customer_id": customer_id, "full_name": row[0]}

# Tool implementations registry (keep/extend your existing one if present)
try:
    tool_implementations
except NameError:
    tool_implementations = {}

tool_implementations["get_customer_name"] = lambda ui, vdb, customer_id: {
    "role": "tool",
    "name": "get_customer_name",
    "content": json.dumps(get_customer_name(customer_id), ensure_ascii=False),
}


___
# Retrieval as a tool

In [8]:
import json

def _invoke_tool(ui, vdb, tool_call):
    tool_name = tool_call.function.name
    ui.rewrite(f"Calling tool {tool_name}")

    tool_args = json.loads(tool_call.function.arguments or "{}")

    tool_impl = tool_implementations[tool_name]
    output = tool_impl(ui, vdb, **tool_args)  # returns {"role":"tool","name":...,"content":...}

    # IMPORTANT: tool message MUST include tool_call_id
    tool_msg = {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": output["content"],
    }
    return tool_msg


def query_llm_with_tools(ui, client, vdb, messages, tools):
    """
    Minimal, correct tool loop:
    - First call (non-stream) to see if model wants tools
    - If tool_calls: append assistant tool_calls -> append tool result with tool_call_id -> call again (stream) for final answer
    - If no tool_calls: just stream answer
    """
    ui.rewrite("[Generating]")

    # 1) NON-STREAM call to decide whether tools are needed
    resp = client.chat.completions.create(
        model="gpt-5.2",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        stream=False,
    )
    assistant_msg = resp.choices[0].message

    tool_calls = getattr(assistant_msg, "tool_calls", None)

    # 2) If no tools requested -> stream normal assistant content
    if not tool_calls:
        stream = client.chat.completions.create(
            model="gpt-5.2",
            messages=messages,
            stream=True,
        )
        return stream

    # 3) Tools requested -> append assistant message WITH tool_calls (required)
    messages.append({
        "role": "assistant",
        "content": assistant_msg.content,
        "tool_calls": [tc.model_dump() for tc in tool_calls],
    })

    # 4) Execute each tool call and append tool results WITH tool_call_id
    for tc in tool_calls:
        tool_msg = _invoke_tool(ui, vdb, tc)
        messages.append(tool_msg)

    # 5) Call model again to produce final answer (streaming is OK here)
    stream = client.chat.completions.create(
        model="gpt-5.2",
        messages=messages,
        stream=True,
    )
    return stream

In [9]:
# keep existing tools and add/override entries
tool_implementations.update({
    "search": add_context,
    "get_customer_name": lambda ui, vdb, customer_id: {
        "role": "tool",
        "name": "get_customer_name",
        "content": json.dumps(get_customer_name(customer_id), ensure_ascii=False),
    }
})

search = {
    "type": "function",
    "function": {
        "name": "search",
        "description": "Use this tool when the user is asking you something you don't know. You should use this tool very often and you can call it many times in a row if necessary.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description":
                        "The query to retrieve the information from the document store using pure embedding similarity search",
                },
            },
            "required": ["query"],
        },
    }
}

get_customer_name_tool = {
  "type": "function",
  "function": {
    "name": "get_customer_name",
    "description": "Look up the customer's full name by customer_id for a personalized greeting. Use only when the user asks about their own products/accounts and provides a customer_id.",
    "parameters": {
      "type": "object",
      "properties": {
        "customer_id": {
          "type": "string",
          "description": "Customer ID in format CUST-#### (e.g., CUST-2002)."
        }
      },
      "required": ["customer_id"]
    }
  }
}

def retrieval_as_tool(paths):
    ui = ChatUI()
    vdb = ingest_documents(ui, paths)
    client, messages = get_oai_client()

    def _retrieval_as_tool(query):
        messages.append({"role": "user", "content": query})
        #stream = query_llm_with_tools(ui, client, vdb, messages, [search])
        stream = query_llm_with_tools(ui, client, vdb, messages, [search, get_customer_name_tool]) ## Calling the model with both tools
        response = display_response(ui, stream)
        messages.append({"role": "assistant", "content": response})

    ui.connect(lambda query: _retrieval_as_tool(query))

In [10]:
retrieval_as_tool(["/content/documents/Bank_Policy_1.md"])

Output()

Output()

Text(value='', description='user: ')

Ingesting documents...
['/content/documents/Bank_Policy_1.md']
/content/documents/Bank_Policy_1.md
processing path: ['/content/documents/Bank_Policy_1.md']
processing path: /content/documents/Bank_Policy_1.md


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Adding context...


/usr/local/lib/python3.12/dist-packages/qdrant_client/common/client_warnings.py:7: UserWarning: `query` method has been deprecated and will be removed in 1.17. Instead, inference can be done internally within regular methods like `query_points` by wrapping data into `models.Document` or `models.Image`.
  warnings.warn(message, category, stacklevel=stacklevel)
